## 1. GPU Setup

In [ ]:
!nvidia-smi

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. Install Dependencies

In [ ]:
!pip install -q ultralytics

from ultralytics import YOLO
import ultralytics
print(f"Ultralytics: {ultralytics.__version__}")

## 3. Dataset Setup

Upload your dataset as a Kaggle dataset. Two options:

**Option A:** `medhack_yolov11` (814 images, YOLO format - ready to use)  
**Option B:** `cocosegmentation-v2-r2` (6592 images, COCO format - needs conversion)

In [ ]:
import os

# UPDATE THIS: Path to your uploaded dataset
DATASET_TYPE = 'yolo'  # 'yolo' or 'coco'
DATASET_PATH = '/kaggle/input/medhack-yolov11'  # or '/kaggle/input/cocosegmentation-v2-r2'

# Verify dataset
print(f"Dataset type: {DATASET_TYPE}")
print(f"Dataset path: {DATASET_PATH}")
print(f"\nDataset contents:")
for item in os.listdir(DATASET_PATH):
    print(f"  - {item}")

## 4. Setup Dataset (YOLO Format)

If using YOLO format (medhack_yolov11), just verify the data.yaml:

In [ ]:
import yaml

if DATASET_TYPE == 'yolo':
    # YOLO format - data.yaml should exist
    data_yaml_path = os.path.join(DATASET_PATH, 'data.yaml')
    
    if os.path.exists(data_yaml_path):
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)
        print("data.yaml found:")
        print(yaml.dump(data_config, default_flow_style=False))
    else:
        print("ERROR: data.yaml not found!")
        print("Creating data.yaml...")
        
        # Create data.yaml
        data_config = {
            'path': DATASET_PATH,
            'train': 'train/images',
            'val': 'valid/images',
            'test': 'test/images',
            'nc': 7,
            'names': ['External Iliac Artery', 'External Iliac Vein', 
                      'Obturator Nerve', 'Ovary', 'Ureter', 
                      'Uterine Artery', 'Uterus']
        }
        
        with open('/kaggle/working/data.yaml', 'w') as f:
            yaml.dump(data_config, f, default_flow_style=False)
        
        data_yaml_path = '/kaggle/working/data.yaml'
        print(f"Created: {data_yaml_path}")
else:
    print("COCO format detected - use COCO conversion cells below")
    data_yaml_path = None

## 5. Advanced Training Configuration

Optimized for maximum accuracy while maintaining real-time performance.

In [ ]:
CONFIG = {
    # Model - YOLOv11s-seg for better accuracy
    'model': 'yolo11s-seg.pt',  # Small model (9M params)
    
    # Training parameters
    'epochs': 150,  # More epochs for better convergence
    'batch': 12,    # P100 can handle this
    'imgsz': 1024,  # Higher resolution for small organ detection
    'patience': 30, # Early stopping patience
    
    # Optimizer settings
    'optimizer': 'AdamW',
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 5.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    
    # Enhanced augmentations for medical imaging
    'hsv_h': 0.015,        # Slight hue variation
    'hsv_s': 0.3,          # Saturation variation
    'hsv_v': 0.2,          # Brightness variation
    'degrees': 10.0,       # Rotation ±10°
    'translate': 0.1,      # Translation
    'scale': 0.3,          # Scaling
    'shear': 0.0,          # No shear (preserves anatomy)
    'perspective': 0.0,    # No perspective (preserves anatomy)
    'flipud': 0.0,         # No vertical flip
    'fliplr': 0.5,         # Horizontal flip
    'mosaic': 0.7,         # Mosaic augmentation
    'mixup': 0.0,          # No mixup (can blur organ boundaries)
    'copy_paste': 0.2,     # Copy-paste for rare organs
    'erasing': 0.0,        # No random erasing
    
    # Loss weights
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    
    # Advanced settings
    'overlap_mask': True,   # Allow overlapping masks
    'mask_ratio': 4,        # Mask downsampling ratio
    'dropout': 0.0,         # No dropout
    
    # System
    'workers': 8,
    'project': '/kaggle/working/runs',
    'name': 'yolov11s_surgical_1024',
    'exist_ok': True,
    'pretrained': True,
    'verbose': True,
    'val': True,
    'plots': True,
    'save': True,
    'save_period': 10,  # Save checkpoint every 10 epochs
}

print("Training Configuration:")
print("=" * 60)
for key, value in CONFIG.items():
    print(f"  {key:20s}: {value}")

## 6. Train YOLOv11s-seg Model

In [ ]:
# Load model
model = YOLO(CONFIG['model'])
print(f"Loaded model: {CONFIG['model']}")
print(f"Parameters: ~9M")

# Start training
print("\n" + "=" * 60)
print("Starting training...")
print("=" * 60 + "\n")

results = model.train(
    data=data_yaml_path,
    epochs=CONFIG['epochs'],
    batch=CONFIG['batch'],
    imgsz=CONFIG['imgsz'],
    patience=CONFIG['patience'],
    optimizer=CONFIG['optimizer'],
    lr0=CONFIG['lr0'],
    lrf=CONFIG['lrf'],
    momentum=CONFIG['momentum'],
    weight_decay=CONFIG['weight_decay'],
    warmup_epochs=CONFIG['warmup_epochs'],
    warmup_momentum=CONFIG['warmup_momentum'],
    warmup_bias_lr=CONFIG['warmup_bias_lr'],
    hsv_h=CONFIG['hsv_h'],
    hsv_s=CONFIG['hsv_s'],
    hsv_v=CONFIG['hsv_v'],
    degrees=CONFIG['degrees'],
    translate=CONFIG['translate'],
    scale=CONFIG['scale'],
    shear=CONFIG['shear'],
    perspective=CONFIG['perspective'],
    flipud=CONFIG['flipud'],
    fliplr=CONFIG['fliplr'],
    mosaic=CONFIG['mosaic'],
    mixup=CONFIG['mixup'],
    copy_paste=CONFIG['copy_paste'],
    erasing=CONFIG['erasing'],
    box=CONFIG['box'],
    cls=CONFIG['cls'],
    dfl=CONFIG['dfl'],
    overlap_mask=CONFIG['overlap_mask'],
    mask_ratio=CONFIG['mask_ratio'],
    dropout=CONFIG['dropout'],
    workers=CONFIG['workers'],
    project=CONFIG['project'],
    name=CONFIG['name'],
    exist_ok=CONFIG['exist_ok'],
    pretrained=CONFIG['pretrained'],
    verbose=CONFIG['verbose'],
    val=CONFIG['val'],
    plots=CONFIG['plots'],
    save=CONFIG['save'],
    save_period=CONFIG['save_period'],
    device=0,
)

print("\n" + "=" * 60)
print("✅ Training complete!")
print("=" * 60)

## 7. Evaluate Model

In [ ]:
# Load best model
best_model_path = f"{CONFIG['project']}/{CONFIG['name']}/weights/best.pt"
print(f"Loading: {best_model_path}")

best_model = YOLO(best_model_path)

# Validate
print("\nValidating on test set...")
metrics = best_model.val(
    data=data_yaml_path,
    split='test',
    imgsz=CONFIG['imgsz'],
    batch=CONFIG['batch'],
    verbose=True,
    plots=True,
)

print("\n" + "=" * 60)
print("Test Results:")
print("=" * 60)
print(f"  Box mAP@50:    {metrics.box.map50:.4f}")
print(f"  Box mAP@50-95: {metrics.box.map:.4f}")
print(f"  Mask mAP@50:   {metrics.seg.map50:.4f}")
print(f"  Mask mAP@50-95:{metrics.seg.map:.4f}")
print("=" * 60)

## 8. Visualize Results

In [ ]:
# Display training curves
from IPython.display import Image, display
import matplotlib.pyplot as plt

results_img = f"{CONFIG['project']}/{CONFIG['name']}/results.png"
if os.path.exists(results_img):
    print("Training Curves:")
    display(Image(filename=results_img))

cm_img = f"{CONFIG['project']}/{CONFIG['name']}/confusion_matrix.png"
if os.path.exists(cm_img):
    print("\nConfusion Matrix:")
    display(Image(filename=cm_img))

## 9. Test Predictions

In [ ]:
import cv2
import numpy as np
import random

# Get test images
test_images_dir = os.path.join(DATASET_PATH, 'test/images')
test_images = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Sample predictions
num_samples = min(6, len(test_images))
samples = random.sample(test_images, num_samples)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, img_file in zip(axes, samples):
    img_path = os.path.join(test_images_dir, img_file)
    results = best_model(img_path, conf=0.5, verbose=False)
    
    annotated = results[0].plot()
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    ax.imshow(annotated)
    ax.set_title(img_file[:25], fontsize=10)
    ax.axis('off')

plt.suptitle('YOLOv11s-seg Predictions (1024px)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Export Models

In [ ]:
# Export to ONNX (faster CPU/GPU inference)
print("Exporting to ONNX...")
best_model.export(
    format='onnx',
    imgsz=640,  # Use 640 for inference (faster)
    simplify=True,
    dynamic=False,
    opset=12,
)
print("✅ ONNX export complete!")

# Optional: Export to TensorRT for maximum speed on NVIDIA GPUs
# Uncomment if you have TensorRT installed
# print("\nExporting to TensorRT...")
# best_model.export(format='engine', imgsz=640, half=True)
# print("✅ TensorRT export complete!")

## 11. Download Models

In [ ]:
import shutil

# Create download directory
output_dir = '/kaggle/working/trained_model_yolov11s'
os.makedirs(output_dir, exist_ok=True)

# Copy weights
weights_dir = f"{CONFIG['project']}/{CONFIG['name']}/weights"
for weight_file in ['best.pt', 'last.pt']:
    src = os.path.join(weights_dir, weight_file)
    if os.path.exists(src):
        shutil.copy(src, output_dir)
        print(f"✅ Copied: {weight_file}")

# Copy ONNX
onnx_file = best_model_path.replace('.pt', '.onnx')
if os.path.exists(onnx_file):
    shutil.copy(onnx_file, output_dir)
    print(f"✅ Copied: best.onnx")

# Copy results
results_dir = f"{CONFIG['project']}/{CONFIG['name']}"
for file in ['results.csv', 'results.png', 'confusion_matrix.png', 'PR_curve.png']:
    src = os.path.join(results_dir, file)
    if os.path.exists(src):
        shutil.copy(src, output_dir)

print(f"\n" + "=" * 60)
print(f"All files saved to: {output_dir}")
print("=" * 60)
print("\nDownloadable files:")
for f in os.listdir(output_dir):
    size = os.path.getsize(os.path.join(output_dir, f)) / (1024*1024)
    print(f"  📦 {f}: {size:.2f} MB")
print("\n💡 Download these files and use with inference_video_smooth_production.py")

---

## Next Steps for Production

1. **Download** `best.pt` and `best.onnx`
2. **Use** `inference_video_smooth_production.py` for real-time inference
3. **Compare** with your nano model:
   - Should see +3-5% mAP improvement
   - Smoother masks
   - Better small organ detection
4. **Test** on live video feed
5. **Fine-tune** confidence threshold based on results

### Expected Performance:
- **MX550 GPU:** 25-35 FPS @ 640px
- **P100 GPU:** 50-70 FPS @ 640px
- **CPU only:** 5-10 FPS (use ONNX)